In [357]:
from openpyxl import load_workbook
wb = load_workbook("bilibili.xlsx")
ws = wb['Sheet1']
sizeOfFile = 500

In [392]:
def get_row(ws, i):
    return [
        int(ws['J' + str(i)].value),
        int(ws['D' + str(i)].value),
        int(ws['F' + str(i)].value),
    ]

In [393]:
data = [get_row(ws, i) for i in range(2,sizeOfFile + 2)]

# 数据预处理

In [394]:
import numpy as np
npData = np.array(data)
npMean = np.mean(npData,axis = 0)
npStd = np.std(npData,axis = 0)
data = (npData - npMean) / npStd #归一化

# K-means：生成随机中心

In [395]:
def randomCenter(center,size):
    return np.random.uniform(-2,2,(center,size))

In [396]:
center = 3
size = 3
initialCenter = randomCenter(center, size)

In [397]:
np.argmin([sum((data[0] - initialCenter[i]) ** 2) for i in range(center)])

0

In [398]:
def update(data, center, initialCenter):
    newList = [[] for _ in range(center)]
    for j in range(sizeOfFile):
        newList[np.argmin([sum((data[j] - initialCenter[i]) ** 2) for i in range(center)])].append(j)
    return newList

In [399]:
def kMean(initialCenter, data, center):
    old = []
    newCenter = initialCenter
    for z in range(1000):
        updated = update(data, center, newCenter)
        if updated == old:
            print(z)
            break
        Center = []
        for i in range(center):
            if updated[i] == []: 
                Center.append(newCenter[i])
                
            else: 
                Center.append(data[updated[i]].mean(axis=0))
        old = updated
        newCenter = Center
        
    return updated

In [400]:
newList = kMean(initialCenter, data, center)

12


In [434]:
def getStdMean(center,ws):
    mean = []
    std = []
    for k in range(center):
        temp = [[int(j.value) for j in ws[i+2][1:]] for i in newList[k]]
        #print(temp)
        mean.append(np.mean(temp,axis = 0))
        std.append(np.std(temp, axis = 0))
    return [mean,std]

In [435]:
mean, std = getStdMean(center, ws)
print(mean[0])

[1.67029885e+06 2.52467672e+02 1.38490503e+08 1.28302155e+03
 3.47855603e+02 1.32155302e+04 7.08005166e+05 1.07894440e+04
 6.91250000e+01 5.99353448e+00]


In [436]:
import xlsxwriter

workbook = xlsxwriter.Workbook('k_Mean_bilibili.xlsx') # 建立文件

worksheet = workbook.add_worksheet() # 建立sheet， 可以work.add_worksheet('employee')来指定sheet名，但中文名会报UnicodeDecodeErro的错误
count = 1
for k in range(center):
    dummy = [i.value for i in ws[1]]
    worksheet.write_row("A" + str(count), dummy)
    count += 1
    for i in newList[k]:
        dummy = [j.value for j in ws[i+2]]
        worksheet.write_row("A" + str(count), dummy)
        count += 1
    mean, std = getStdMean(center, ws)
    dummy = []
    dummy.append("关注mean: " + str(mean[k][8]))
    dummy.append("关注std: " + str(std[k][8]))
    dummy.append("总播放量mean: " + str(mean[k][2]))
    dummy.append("总播放量std: " + str(std[k][2]))
    dummy.append("视频数mean: " + str(mean[k][4]))
    dummy.append("视频数std: " + str(std[k][4]))
    worksheet.write_row("A" + str(count), dummy)
    count += 3
 
# 将数据写入单元格序列
workbook.close()

# 输出分类

In [259]:
nameList = [ws['A' + str(i+2)].value for i in newList[3] ]

In [260]:
nameList = []
for j in range(center):
    nameList.append([ws['A' + str(i+2)].value for i in newList[j] ])

In [277]:
for i in nameList:
    print((i))
    print("\n")

['敖厂长', '共青团中央', '翔翔大作战', '华农兄弟', '暴走漫画', '中国BOY超级大猩猩', '指法芬芳张大仙', '哔哩哔哩电影', '朱一旦的枯燥生活', '徐大sao', '哔哩哔哩英雄联盟赛事', '靠脸吃饭的徐大王', '起小点是大腿']


['papi酱', '凉风Kaze', '周六野Zoey', '机智的党妹', '罗翔说刑法', '李子柒', '伊丽莎白鼠', '某幻君', '渗透之C君', '我是郭杰瑞', '信誓蛋蛋', '徐大虾咯']


['哔哩哔哩漫画', '老番茄', '敬汉卿', '哔哩哔哩大会员', 'LexBurner']


['老师好我叫何同学', '木鱼水心', '美食作家王刚R', '纳豆奶奶', '逍遥散人', '张召忠', '绵羊料理', '爱做饭的芋头SAMA', '记录生活的蛋黄派', '大祥哥来了', '日食记', '手工耿', '央视新闻', '硬核的半佛仙人', '歪果仁研究协会', '花少北丶', '努力的Lorre']


['哔哩哔哩纪录片', '观察者网', '柚子木字幕组']




# （播放量/视频数） -  粉丝数 -> 计算粉丝数和实际活跃用户差值

In [348]:
diffData = [get_row(ws, i) for i in range(2,sizeOfFile + 2)]

In [349]:
sortUp = []

for index, values in enumerate(diffData):
    temp = []
    temp.append(values[1]/values[2]-values[0])
    temp.append(index)
    sortUp.append(temp)
    
sortUp.sort(reverse = 1)

In [350]:
import xlsxwriter

workbook = xlsxwriter.Workbook('Diff_bilibili.xlsx') # 建立文件

worksheet = workbook.add_worksheet() # 建立sheet， 可以work.add_worksheet('employee')来指定sheet名，但中文名会报UnicodeDecodeErro的错误
count = 1
dummy = [i.value for i in ws[1]]
dummy.append("（播放量/视频数） - 粉丝数")
worksheet.write_row("A" + str(count), dummy)
count += 1
for i,index in sortUp:
    dummy = [i.value for i in ws[index + 2]]
    dummy.append(i)
    worksheet.write_row("A" + str(count), dummy)
    count += 1 
# 将数据写入单元格序列
workbook.close()

# 粉丝数*视频数/播放量

In [354]:
diffData = [get_row(ws, i) for i in range(2,sizeOfFile + 2)]

In [355]:
newSortUp = []

for index, values in enumerate(diffData):
    temp = []
    temp.append(values[1]/(values[0]*values[2]))
    temp.append(index)
    newSortUp.append(temp)
    
newSortUp.sort(reverse = 1)

In [356]:
import xlsxwriter

workbook = xlsxwriter.Workbook('平均视频观看量 _bilibili.xlsx') # 建立文件

worksheet = workbook.add_worksheet() # 建立sheet， 可以work.add_worksheet('employee')来指定sheet名，但中文名会报UnicodeDecodeErro的错误
count = 1
dummy = [i.value for i in ws[1]]
dummy.append("播放量/（粉丝数*视频数）")
worksheet.write_row("A" + str(count), dummy)
count += 1
for i,index in newSortUp:
    dummy = [i.value for i in ws[index + 2]]
    dummy.append(i)
    worksheet.write_row("A" + str(count), dummy)
    count += 1 
# 将数据写入单元格序列
workbook.close()